In [1]:
import pandas as pd

In [2]:
tri_data = pd.read_csv('tridata.csv')

# C)

In [3]:
tri_data = tri_data.drop(['EVENT_NO_STOP', 'GPS_SATELLITES', 'GPS_HDOP'], axis='columns')

In [4]:
tri_data.head()

,EVENT_NO_TRIP,OPD_DATE,VEHICLE_ID,METERS,ACT_TIME,GPS_LONGITUDE,GPS_LATITUDE
0,259172515,15FEB2023:00:00:00,4223,40,20469,-122.648137,45.493082
1,259172515,15FEB2023:00:00:00,4223,48,20474,-122.648240,45.493070
2,259172515,15FEB2023:00:00:00,4223,57,20479,-122.648352,45.493123
3,259172515,15FEB2023:00:00:00,4223,73,20484,-122.648385,45.493262
4,259172515,15FEB2023:00:00:00,4223,112,20489,-122.648347,45.493582


In [5]:
tri_data = pd.read_csv('tridata.csv', usecols=['EVENT_NO_TRIP', 'OPD_DATE', 'VEHICLE_ID', 'METERS', 'ACT_TIME', 'GPS_LONGITUDE', 'GPS_LATITUDE'])

In [6]:
tri_data.head()

,EVENT_NO_TRIP,OPD_DATE,VEHICLE_ID,METERS,ACT_TIME,GPS_LONGITUDE,GPS_LATITUDE
0,259172515,15FEB2023:00:00:00,4223,40,20469,-122.648137,45.493082
1,259172515,15FEB2023:00:00:00,4223,48,20474,-122.648240,45.493070
2,259172515,15FEB2023:00:00:00,4223,57,20479,-122.648352,45.493123
3,259172515,15FEB2023:00:00:00,4223,73,20484,-122.648385,45.493262
4,259172515,15FEB2023:00:00:00,4223,112,20489,-122.648347,45.493582


# D)

In [7]:
tri_data.insert(5, 'TIMESTAMP', 0)
tri_data.head()

,EVENT_NO_TRIP,OPD_DATE,VEHICLE_ID,METERS,ACT_TIME,TIMESTAMP,GPS_LONGITUDE,GPS_LATITUDE
0,259172515,15FEB2023:00:00:00,4223,40,20469,0,-122.648137,45.493082
1,259172515,15FEB2023:00:00:00,4223,48,20474,0,-122.648240,45.493070
2,259172515,15FEB2023:00:00:00,4223,57,20479,0,-122.648352,45.493123
3,259172515,15FEB2023:00:00:00,4223,73,20484,0,-122.648385,45.493262
4,259172515,15FEB2023:00:00:00,4223,112,20489,0,-122.648347,45.493582


In [9]:
from datetime import datetime

In [10]:
def decode_func(tri_data: pd.DataFrame):
    mod_data = tri_data.copy()
    opd_date = tri_data['OPD_DATE']
    act_time = tri_data['ACT_TIME']
    date_format = "%d%b%Y:%H:%M:%S"
    opd_sec: pd.Timestamp = pd.to_datetime(pd.to_datetime(opd_date, format=date_format), unit='s')
    td: pd.Timedelta = pd.to_timedelta(act_time, unit='sec')

    mod_data['TIMESTAMP'] = opd_sec + td
    return mod_data

In [11]:
new_data = tri_data.apply(decode_func, axis=1)

In [12]:
new_data.head()

,EVENT_NO_TRIP,OPD_DATE,VEHICLE_ID,METERS,ACT_TIME,TIMESTAMP,GPS_LONGITUDE,GPS_LATITUDE
0,259172515,15FEB2023:00:00:00,4223,40,20469,2023-02-15 05:41:09,-122.648137,45.493082
1,259172515,15FEB2023:00:00:00,4223,48,20474,2023-02-15 05:41:14,-122.648240,45.493070
2,259172515,15FEB2023:00:00:00,4223,57,20479,2023-02-15 05:41:19,-122.648352,45.493123
3,259172515,15FEB2023:00:00:00,4223,73,20484,2023-02-15 05:41:24,-122.648385,45.493262
4,259172515,15FEB2023:00:00:00,4223,112,20489,2023-02-15 05:41:29,-122.648347,45.493582


# E)

In [13]:
new_data = new_data.drop(['OPD_DATE', 'ACT_TIME'], axis=1)

# F)

In [14]:
new_data.insert(4, 'SPEED', 0)
new_data.insert(4, 'dMETERS', new_data['METERS'].diff())
new_data.insert(4, 'dTIMESTAMP', new_data['TIMESTAMP'].diff())

In [15]:
new_data = new_data.drop(index=0)

In [16]:
new_data.head()

,EVENT_NO_TRIP,VEHICLE_ID,METERS,TIMESTAMP,dTIMESTAMP,dMETERS,SPEED,GPS_LONGITUDE,GPS_LATITUDE
1,259172515,4223,48,2023-02-15 05:41:14,0 days 00:00:05,8.0,0,-122.648240,45.493070
2,259172515,4223,57,2023-02-15 05:41:19,0 days 00:00:05,9.0,0,-122.648352,45.493123
3,259172515,4223,73,2023-02-15 05:41:24,0 days 00:00:05,16.0,0,-122.648385,45.493262
4,259172515,4223,112,2023-02-15 05:41:29,0 days 00:00:05,39.0,0,-122.648347,45.493582
5,259172515,4223,159,2023-02-15 05:41:34,0 days 00:00:05,47.0,0,-122.648357,45.494003


In [17]:
newer_data = new_data.copy()
newer_data['SPEED'] = newer_data.apply((lambda newer_data: newer_data['dMETERS']/newer_data['dTIMESTAMP'].total_seconds()), axis=1)

In [18]:
newer_data = newer_data.drop(['dTIMESTAMP', 'dMETERS'], axis=1)

In [19]:
newer_data.head()

,EVENT_NO_TRIP,VEHICLE_ID,METERS,TIMESTAMP,SPEED,GPS_LONGITUDE,GPS_LATITUDE
1,259172515,4223,48,2023-02-15 05:41:14,1.6,-122.648240,45.493070
2,259172515,4223,57,2023-02-15 05:41:19,1.8,-122.648352,45.493123
3,259172515,4223,73,2023-02-15 05:41:24,3.2,-122.648385,45.493262
4,259172515,4223,112,2023-02-15 05:41:29,7.8,-122.648347,45.493582
5,259172515,4223,159,2023-02-15 05:41:34,9.4,-122.648357,45.494003


In [20]:
newer_data.describe()

,EVENT_NO_TRIP,VEHICLE_ID,METERS,TIMESTAMP,SPEED,GPS_LONGITUDE,GPS_LATITUDE
count,160.0,160.0,160.000000,160,160.000000,160.000000,160.000000
mean,259172515.0,4223.0,3611.368750,2023-02-15 05:48:24.374999808,7.227206,-122.666758,45.514778
min,259172515.0,4223.0,48.000000,2023-02-15 05:41:14,0.000000,-122.677585,45.493070
25%,259172515.0,4223.0,1993.750000,2023-02-15 05:44:37.750000128,3.800000,-122.676085,45.503697
50%,259172515.0,4223.0,4196.000000,2023-02-15 05:47:56.500000,6.400000,-122.670099,45.518003
75%,259172515.0,4223.0,5212.000000,2023-02-15 05:51:35.249999872,10.850000,-122.660809,45.523219
max,259172515.0,4223.0,5918.000000,2023-02-15 05:57:19,17.400000,-122.647997,45.528065
std,0.0,0.0,1794.287464,NaN,4.420604,0.010017,0.010459
